In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path
import os
import cv2

calibrationfolder = 'Calibration_20250704'
rootfolderpath = Path('/Users/kaitohara-lee/Library/CloudStorage/OneDrive-Queen\'sUniversity/School/UTokyo/Project')

# Print number of files in the calibration folder/Images
calibrationfolderpath = rootfolderpath / calibrationfolder
calibrationimagespath = calibrationfolderpath / 'Images'
calibrationimages = list(calibrationimagespath.glob('*.tga'))
print(f'Number of calibration images: {len(calibrationimages)}')

Number of calibration images: 20


In [4]:
# Loop through the images and split them left and right and print shape of each image
for imagefile in calibrationimages:
    image = Image.open(imagefile)
    imagearr = np.array(image)
    print(f'Image file: {imagefile.name}')
    print('Image size:', image.size)
    print('Image array shape:', np.shape(imagearr))

    left = imagearr[::2, :, :]  # Odd rows extracted
    right = imagearr[1::2, :, :]  # Even rows extracted

    print('Left image shape:', left.shape)
    print('Right image shape:', right.shape)
    print('-' * 40)
    
    # Save the left and right images in folders called 'Left' and 'Right'
    left_folder = calibrationfolderpath / 'Left'
    right_folder = calibrationfolderpath / 'Right'
    left_folder.mkdir(exist_ok=True)
    right_folder.mkdir(exist_ok=True)
    left_image = Image.fromarray(left)
    right_image = Image.fromarray(right)
    # save the images with _left and _right suffixes
    left_image.save(left_folder / f'{imagefile.stem}_left.tga')
    right_image.save(right_folder / f'{imagefile.stem}_right.tga')
    
    

Image file: 02 9.tga
Image size: (1920, 1080)
Image array shape: (1080, 1920, 4)
Left image shape: (540, 1920, 4)
Right image shape: (540, 1920, 4)
----------------------------------------
Image file: 02 17.tga
Image size: (1920, 1080)
Image array shape: (1080, 1920, 4)
Left image shape: (540, 1920, 4)
Right image shape: (540, 1920, 4)
----------------------------------------
Image file: 02 16.tga
Image size: (1920, 1080)
Image array shape: (1080, 1920, 4)
Left image shape: (540, 1920, 4)
Right image shape: (540, 1920, 4)
----------------------------------------
Image file: 02 8.tga
Image size: (1920, 1080)
Image array shape: (1080, 1920, 4)
Left image shape: (540, 1920, 4)
Right image shape: (540, 1920, 4)
----------------------------------------
Image file: 02 14.tga
Image size: (1920, 1080)
Image array shape: (1080, 1920, 4)
Left image shape: (540, 1920, 4)
Right image shape: (540, 1920, 4)
----------------------------------------
Image file: 02 15.tga
Image size: (1920, 1080)
Image

In [23]:
# Create a "NN" folder to save the nearest neighbor interpolated images
nn_folder = calibrationfolderpath / 'NN'
nn_folder.mkdir(exist_ok=True)

# Create subfolders for 1920x1080 and 1280x720 for left and right images
nn_1080_folder = nn_folder / '1920x1080'
nn_720_folder = nn_folder / '1280x720'
nn_left_folder_1080 = nn_1080_folder / 'left'
nn_right_folder_1080 = nn_1080_folder / 'right'
nn_left_folder_720 = nn_720_folder / 'left'
nn_right_folder_720 = nn_720_folder / 'right'
os.makedirs(nn_left_folder_1080, exist_ok=True)
os.makedirs(nn_right_folder_1080, exist_ok=True)
os.makedirs(nn_left_folder_720, exist_ok=True)
os.makedirs(nn_right_folder_720, exist_ok=True)

# loop through left and right images and resize them to 1920x1080 and 1280x720 using nearest neighbor interpolation
left_folder = calibrationfolderpath / 'Left'
right_folder = calibrationfolderpath / 'Right'

for left,right in zip(left_folder.glob('*.tga'), right_folder.glob('*.tga')):
    leftim = Image.open(left)
    rightim = Image.open(right)
    leftarr = np.array(leftim)
    rightarr = np.array(rightim)

    # Nearest neighbor interpolation to 1920x1080
    left_interp_1080 = cv2.resize(leftarr, (1920, 1080), interpolation=cv2.INTER_NEAREST)
    right_interp_1080 = cv2.resize(rightarr, (1920, 1080), interpolation=cv2.INTER_NEAREST)

    # Save the interpolated images with _1080 suffix
    Image.fromarray(left_interp_1080).save(nn_left_folder_1080 / f'{left.stem}_1080.tga')
    Image.fromarray(right_interp_1080).save(nn_right_folder_1080 / f'{right.stem}_1080.tga')

    # Nearest neighbor interpolation to 1280x720
    left_interp_720 = cv2.resize(leftarr, (1280, 720), interpolation=cv2.INTER_NEAREST)
    right_interp_720 = cv2.resize(rightarr, (1280, 720), interpolation=cv2.INTER_NEAREST)  
    
    # Save the interpolated images with _720 suffix
    Image.fromarray(left_interp_720).save(nn_left_folder_720 / f'{left.stem}_720.tga')
    Image.fromarray(right_interp_720).save(nn_right_folder_720 / f'{right.stem}_720.tga')

In [24]:
# Create a "BL" folder to save the bilinear interpolated images
bl_folder = calibrationfolderpath / 'BL'
bl_folder.mkdir(exist_ok=True)

# Create subfolders for 1920x1080 and 1280x720 for left and right images
bl_1080_folder = bl_folder / '1920x1080'
bl_720_folder = bl_folder / '1280x720'
bl_left_folder_1080 = bl_1080_folder / 'left'
bl_right_folder_1080 = bl_1080_folder / 'right'
bl_left_folder_720 = bl_720_folder / 'left'
bl_right_folder_720 = bl_720_folder / 'right'
os.makedirs(bl_left_folder_1080, exist_ok=True)
os.makedirs(bl_right_folder_1080, exist_ok=True)
os.makedirs(bl_left_folder_720, exist_ok=True)
os.makedirs(bl_right_folder_720, exist_ok=True)

# Define input folders
left_folder = calibrationfolderpath / 'Left'
right_folder = calibrationfolderpath / 'Right'

# Loop through left and right images and resize with bilinear interpolation
for left, right in zip(left_folder.glob('*.tga'), right_folder.glob('*.tga')):
    leftim = Image.open(left)
    rightim = Image.open(right)
    leftarr = np.array(leftim)
    rightarr = np.array(rightim)

    # Bilinear interpolation to 1920x1080
    left_interp_1080 = cv2.resize(leftarr, (1920, 1080), interpolation=cv2.INTER_LINEAR)
    right_interp_1080 = cv2.resize(rightarr, (1920, 1080), interpolation=cv2.INTER_LINEAR)

    # Save the interpolated images with _1080 suffix
    Image.fromarray(left_interp_1080).save(bl_left_folder_1080 / f'{left.stem}_1080.tga')
    Image.fromarray(right_interp_1080).save(bl_right_folder_1080 / f'{right.stem}_1080.tga')

    # Bilinear interpolation to 1280x720
    left_interp_720 = cv2.resize(leftarr, (1280, 720), interpolation=cv2.INTER_LINEAR)
    right_interp_720 = cv2.resize(rightarr, (1280, 720), interpolation=cv2.INTER_LINEAR)  
    
    # Save the interpolated images with _720 suffix
    Image.fromarray(left_interp_720).save(bl_left_folder_720 / f'{left.stem}_720.tga')
    Image.fromarray(right_interp_720).save(bl_right_folder_720 / f'{right.stem}_720.tga')


In [25]:
# Create a "BC" folder to save the bicubic interpolated images
bc_folder = calibrationfolderpath / 'BC'
bc_folder.mkdir(exist_ok=True)

# Create subfolders for 1920x1080 and 1280x720 for left and right images
bc_1080_folder = bc_folder / '1920x1080'
bc_720_folder = bc_folder / '1280x720'
bc_left_folder_1080 = bc_1080_folder / 'left'
bc_right_folder_1080 = bc_1080_folder / 'right'
bc_left_folder_720 = bc_720_folder / 'left'
bc_right_folder_720 = bc_720_folder / 'right'
os.makedirs(bc_left_folder_1080, exist_ok=True)
os.makedirs(bc_right_folder_1080, exist_ok=True)
os.makedirs(bc_left_folder_720, exist_ok=True)
os.makedirs(bc_right_folder_720, exist_ok=True)

# Define input folders
left_folder = calibrationfolderpath / 'Left'
right_folder = calibrationfolderpath / 'Right'

# Loop through left and right images and resize with bicubic interpolation
for left, right in zip(left_folder.glob('*.tga'), right_folder.glob('*.tga')):
    leftim = Image.open(left)
    rightim = Image.open(right)
    leftarr = np.array(leftim)
    rightarr = np.array(rightim)

    # Bicubic interpolation to 1920x1080
    left_interp_1080 = cv2.resize(leftarr, (1920, 1080), interpolation=cv2.INTER_CUBIC)
    right_interp_1080 = cv2.resize(rightarr, (1920, 1080), interpolation=cv2.INTER_CUBIC)

    # Save the interpolated images with _1080 suffix
    Image.fromarray(left_interp_1080).save(bc_left_folder_1080 / f'{left.stem}_1080.tga')
    Image.fromarray(right_interp_1080).save(bc_right_folder_1080 / f'{right.stem}_1080.tga')

    # Bicubic interpolation to 1280x720
    left_interp_720 = cv2.resize(leftarr, (1280, 720), interpolation=cv2.INTER_CUBIC)
    right_interp_720 = cv2.resize(rightarr, (1280, 720), interpolation=cv2.INTER_CUBIC)  
    
    # Save the interpolated images with _720 suffix
    Image.fromarray(left_interp_720).save(bc_left_folder_720 / f'{left.stem}_720.tga')
    Image.fromarray(right_interp_720).save(bc_right_folder_720 / f'{right.stem}_720.tga')

In [26]:
# Create a "LZ" folder to save the Lanczos interpolated images
lz_folder = calibrationfolderpath / 'LZ'
lz_folder.mkdir(exist_ok=True)

# Create subfolders for 1920x1080 and 1280x720 for left and right images
lz_1080_folder = lz_folder / '1920x1080'
lz_720_folder = lz_folder / '1280x720'
lz_left_folder_1080 = lz_1080_folder / 'left'
lz_right_folder_1080 = lz_1080_folder / 'right'
lz_left_folder_720 = lz_720_folder / 'left'
lz_right_folder_720 = lz_720_folder / 'right'
os.makedirs(lz_left_folder_1080, exist_ok=True)
os.makedirs(lz_right_folder_1080, exist_ok=True)
os.makedirs(lz_left_folder_720, exist_ok=True)
os.makedirs(lz_right_folder_720, exist_ok=True)

# Define input folders
left_folder = calibrationfolderpath / 'Left'
right_folder = calibrationfolderpath / 'Right'

# Loop through left and right images and resize with Lanczos interpolation
for left, right in zip(left_folder.glob('*.tga'), right_folder.glob('*.tga')):
    leftim = Image.open(left)
    rightim = Image.open(right)
    leftarr = np.array(leftim)
    rightarr = np.array(rightim)

    # Lanczos interpolation to 1920x1080
    left_interp_1080 = cv2.resize(leftarr, (1920, 1080), interpolation=cv2.INTER_LANCZOS4)
    right_interp_1080 = cv2.resize(rightarr, (1920, 1080), interpolation=cv2.INTER_LANCZOS4)

    # Save the interpolated images with _1080 suffix
    Image.fromarray(left_interp_1080).save(lz_left_folder_1080 / f'{left.stem}_1080.tga')
    Image.fromarray(right_interp_1080).save(lz_right_folder_1080 / f'{right.stem}_1080.tga')

    # Lanczos interpolation to 1280x720
    left_interp_720 = cv2.resize(leftarr, (1280, 720), interpolation=cv2.INTER_LANCZOS4)
    right_interp_720 = cv2.resize(rightarr, (1280, 720), interpolation=cv2.INTER_LANCZOS4)  
    
    # Save the interpolated images with _720 suffix
    Image.fromarray(left_interp_720).save(lz_left_folder_720 / f'{left.stem}_720.tga')
    Image.fromarray(right_interp_720).save(lz_right_folder_720 / f'{right.stem}_720.tga')
